# Spatial Dropout (Channel-wise Dropout) — En Temelden Yazım + Modele Entegrasyon

Bu notebook iki şeyi bitirir:
1) **Spatial Dropout’u sıfırdan** (mask, broadcast, scaling) yazarız.
2) Bunu **CNN / Residual blok** içine doğru yere koyup entegre ederiz.

Hedef: PyTorch’ta `nn.Dropout2d` ne yapıyorsa onu *tam anlayıp* kendin yazabilecek seviyeye gelmek.


## 0) Spatial Dropout tam olarak ne yapıyor?

- Girdi tensörü: **x ∈ R[B, C, H, W]**
- Her örnek için her kanal ya tutulur ya tamamen sıfırlanır.
- **Klasik dropout** element-wise maske üretir: `[B,C,H,W]`
- **Spatial dropout** channel-wise maske üretir: `[B,C,1,1]` → H,W’ye broadcast

Bu yüzden CNN’de kanal co-adaptation (kanalların birbirine aşırı bağımlılığı) azalır.


## 1) En temel matematik

- Drop probability: **p**
- Keep probability: **q = 1 - p**

Maske:
$$m \sim Bernoulli(q),\; m \in \{0,1\}^{B\times C\times 1\times 1}$$

Uygulama (inverted dropout):
$$y = \frac{x \odot m}{q}$$

İnverted scaling olayı şunu sağlar:
- Train sırasında beklenen değer korunur (yaklaşık) → eval’de ek bir ölçekleme gerekmez.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

B, C, H, W = 2, 4, 5, 5
x = torch.randn(B, C, H, W)
x.shape

torch.Size([2, 4, 5, 5])

## 2) Spatial Dropout’u sıfırdan yazalım (fonksiyon seviyesi)

Aşağıdaki fonksiyon en minimal haliyle şunu yapar:
1) `mask` üretir: `[B, C, 1, 1]`
2) `x * mask` uygular
3) `q` ile böler (inverted dropout)

Not: Bu fonksiyon **sadece training modunda** çalıştırılmalı.


In [2]:
def spatial_dropout2d(x: torch.Tensor, p: float, training: bool) -> torch.Tensor:
    """Channel-wise dropout (Spatial Dropout / Dropout2d) from scratch.
    x: [B,C,H,W]
    p: drop prob
    training: if False -> identity
    """
    if not (0.0 <= p < 1.0):
        raise ValueError("p must be in [0, 1).")
    if (not training) or p == 0.0:
        return x

    q = 1.0 - p
    # mask: [B, C, 1, 1]  (broadcast to H,W)
    mask = torch.empty((x.size(0), x.size(1), 1, 1), device=x.device, dtype=x.dtype).bernoulli_(q)
    return x * mask / q

y = spatial_dropout2d(x, p=0.5, training=True)
y.shape

torch.Size([2, 4, 5, 5])

### 2.1) Kanalın komple düştüğünü nasıl doğrularız?

Bir kanal komple 0’a gittiyse o kanalın `abs().mean()` değeri 0 olur.


In [3]:
channel_energy = y[0].abs().mean(dim=(1,2))
channel_energy  # 0 olan kanal -> komple düştü

tensor([1.7126, 0.0000, 0.0000, 1.5186])

## 3) Bunu düzgün bir nn.Module yapalım

Bu noktada iki kritik şey var:
- `self.training` flag'i doğru çalışmalı (`model.train()` / `model.eval()`)
- p=0 ise identity


In [4]:
class SpatialDropout2D(nn.Module):
    def __init__(self, p: float = 0.1):
        super().__init__()
        if not (0.0 <= p < 1.0):
            raise ValueError("p must be in [0, 1).")
        self.p = float(p)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return spatial_dropout2d(x, p=self.p, training=self.training)

sd = SpatialDropout2D(p=0.5)
sd.train()
y_train = sd(x)
sd.eval()
y_eval = sd(x)

torch.allclose(x, y_eval), y_train.shape

(True, torch.Size([2, 4, 5, 5]))

## 4) Pytorch `nn.Dropout2d` ile aynı mı?

Tam birebir aynı davranış için bazı PyTorch sürümlerinde `Dropout2d` maskeyi örnek/kanal bazlı uygular.
Bizimki de aynı fikri uygular.

Aşağıda çıktıların *birebir aynı olması* şart değil (random mask), ama davranışın tipi aynı olmalı:
- Bazı kanallar tamamen 0
- Eval modunda hiç dokunmuyor


In [5]:
drop2d = nn.Dropout2d(p=0.5)
drop2d.train()
y_builtin = drop2d(x)

print("custom channel energy:", y_train[0].abs().mean(dim=(1,2)))
print("builtin channel energy:", y_builtin[0].abs().mean(dim=(1,2)))

custom channel energy: tensor([0.0000, 0.0000, 2.0364, 1.5186])
builtin channel energy: tensor([0.0000, 0.0000, 0.0000, 1.5186])


## 5) Modele entegrasyon: NEREYE KOYACAĞIZ?

En pratik kural:
**Conv → Norm → Activation → Spatial Dropout**

Neden activation sonrası?
- Norm istatistiklerini dropout ile bozmak istemezsin.
- Feature üretildikten sonra bazı kanalları kapatıp robustluk kazandırırsın.

Örnek blok:
- Conv → BN → SiLU → SD


In [6]:
class ConvBNActSD(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1, act="silu", sd_p=0.1):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        self.act = nn.SiLU(inplace=True) if act == "silu" else nn.ReLU(inplace=True)
        self.sd = SpatialDropout2D(p=sd_p)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        x = self.sd(x)
        return x

blk = ConvBNActSD(3, 16, sd_p=0.2)
blk.train()
out = blk(torch.randn(2,3,32,32))
out.shape

torch.Size([2, 16, 32, 32])

## 6) Residual blok içine entegrasyon

Senin DropPath tarafıyla birlikte düşüneceğin yer burası.

Tipik residual:
$$y = x + F(x)$$

Spatial Dropout’u `F(x)` içinde uygularsın.
Genelde:
- ikinci conv sonrası
- veya attention sonrası
- veya blok çıkışına yakın

Aşağıda basit bir ResBlock var:


In [7]:
class BasicResBlockSD(nn.Module):
    def __init__(self, c, sd_p=0.1):
        super().__init__()
        self.conv1 = nn.Conv2d(c, c, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(c)
        self.act = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(c, c, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(c)
        self.sd = SpatialDropout2D(p=sd_p)

    def forward(self, x):
        identity = x
        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        # SD'yi branch içinde uygula
        out = self.sd(out)
        out = out + identity
        out = self.act(out)
        return out

rb = BasicResBlockSD(16, sd_p=0.2)
rb.train()
z = rb(torch.randn(2,16,32,32))
z.shape

torch.Size([2, 16, 32, 32])

## 7) Tam model örneği (küçük ama gerçekçi)

Aşağıdaki model:
- Stem conv
- 2 tane residual blok
- Global average pooling
- Linear classifier

Spatial Dropout’u hem ConvBNAct içinde hem residual içinde kullanıyoruz.


In [8]:
class TinyCNNWithSpatialDropout(nn.Module):
    def __init__(self, num_classes=10, sd_p=0.1):
        super().__init__()
        self.stem = ConvBNActSD(3, 32, k=3, s=1, p=1, act="silu", sd_p=sd_p)
        self.proj = nn.Conv2d(32, 64, 3, stride=2, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(64)
        self.act = nn.SiLU(inplace=True)
        self.block1 = BasicResBlockSD(64, sd_p=sd_p)
        self.block2 = BasicResBlockSD(64, sd_p=sd_p)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.act(self.bn(self.proj(x)))
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool(x).flatten(1)
        x = self.fc(x)
        return x

m = TinyCNNWithSpatialDropout(num_classes=5, sd_p=0.2)
m.train()
logits = m(torch.randn(2,3,64,64))
logits.shape

torch.Size([2, 5])

## 8) Training loop içine nasıl girer?

Spatial Dropout, `model.train()` iken çalışır.
Yani entegrasyon ekstra bir şey istemez:
- Train: `model.train()`
- Eval: `model.eval()`

Aşağıda minimal bir eğitim iskeleti var (dummy data ile).


In [9]:
def train_step(model, optimizer, x, y):
    model.train()
    optimizer.zero_grad(set_to_none=True)
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    loss.backward()
    optimizer.step()
    return float(loss.detach())

def eval_step(model, x, y):
    model.eval()
    with torch.no_grad():
        logits = model(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
    return float(loss), float(acc)

model = TinyCNNWithSpatialDropout(num_classes=10, sd_p=0.2)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)

x_batch = torch.randn(8,3,64,64)
y_batch = torch.randint(0,10,(8,))

loss = train_step(model, opt, x_batch, y_batch)
val_loss, val_acc = eval_step(model, x_batch, y_batch)
loss, val_loss, val_acc

(2.439744710922241, 2.304030656814575, 0.125)

## 9) Pratik ayarlar (senin setup için net öneri)

- Başlangıç: **p = 0.05 – 0.15**
- Overfit görürsen: 0.2
- DropPath de kullanıyorsan (sen kullanıyorsun):
  - ikisini aynı anda yüksek yapma
  - genelde DropPath ana regularizer olur, SD düşük kalır

Yerleşim:
- Conv-BN-Act sonrası
- Attention sonrası
- Residual branch içinde (skip yoluna değil)


## 10) Mini checklist (bunu repo’da yap)

- [ ] `SpatialDropout2D` modülünü `regularization/` altına koy
- [ ] Basit ablation: p=0.0 / 0.1 / 0.2
- [ ] DropPath ile birlikteyken SD’yi 0.05–0.1’e çek
- [ ] Eğitim loglarında overfit azaldı mı bak

Hazır olduğunda bir sonraki konu: **Stochastic Depth**.
